In [2]:
!jupyter nbconvert --to python ./sentinel2-sattelite-images-classification-solution-design-WIP.ipynb

[NbConvertApp] Converting notebook ./sentinel2-sattelite-images-classification-solution-design-WIP.ipynb to python
[NbConvertApp] Writing 11146 bytes to ./sentinel2-sattelite-images-classification-solution-design-WIP.py


In [138]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from geojson import Polygon
from datetime import date
import json
import folium

with open('sentinel_user_login.json', 'r') as fp:
    LOGIN_INFO = json.load(fp)
USER_NAME, PASSWORD = list(LOGIN_INFO.values())
# USER_NAME, PASSWORD = '<YourUserName>', '<YourPassword>'

# define a map polygon
coordinates = [[
    (18.025318309126252, 48.25365988314252),
    (17.945005267006067, 48.18825958525136),
    (18.043165651819628, 48.18825958525136),
    (18.025318309126252, 48.25365988314252),
    (18.025318309126252, 48.25365988314252)
]]
geojson = Polygon(coordinates=coordinates)

# connect to the API
api = SentinelAPI(USER_NAME, PASSWORD, 'https://scihub.copernicus.eu/dhus')

# search by polygon, time, and Hub query keywords
footprint = geojson_to_wkt(geojson)
dates = (date(2017, 6, 10), date(2017, 8, 24))
dates = (date(2018, 4, 1), date(2018, 4, 11))

# June to July maps
products = api.query(footprint,
                     date=dates,
                     platformname='Sentinel-2',
#                      producttype='S2MSI1C',
                     producttype='S2MSI2A',
                     area_relation='Intersects',  # area of interest is inside footprint
#                      area_relation='Contains',  # area of interest is inside footprint
#                      area_relation='IsWithin',  # area of interest is inside footprint
                     cloudcoverpercentage=(0, 40))

product_id = list(products.keys())[0]  # June

# GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
geo_df = api.to_geodataframe(products).loc[[product_id]][['geometry']]

def add_choropleth(mapobj, gdf, fill_color='Blue', fill_opacity=0.6,
                   line_opacity=0.2, num_classes=5):
    # Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    
    # Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON,
    # the data as the GeoDataFrame, the columns as the user-specified id field and and value field.
    # key_on field refers to the id field within the GeoJSON string
    mapobj.choropleth(
        geo_data=gdf_wgs84.to_json(),
        fill_color=fill_color,
        fill_opacity=fill_opacity,
        line_opacity=line_opacity,  
    )
    return mapobj

slovakia_map = folium.Map(np.mean(coordinates[0], 0).tolist()[::-1], zoom_start=8)
slovakia_map = add_choropleth(slovakia_map, geo_df)
slovakia_map

In [139]:
# download all results from the search
# api.download_all(products)
api.download(id=product_id, directory_path='../data')

Downloading: 100%|██████████| 1.20G/1.20G [04:12<00:00, 4.76MB/s]
MD5 checksumming: 100%|██████████| 1.20G/1.20G [00:02<00:00, 556MB/s]


{'id': '2e938509-84f3-4ae1-b2c4-0da5680c9588',
 'title': 'S2B_MSIL2A_20180408T095029_N0207_R079_T33UYP_20180408T115858',
 'size': 1203884722,
 'md5': '448BF75624F75C34FB668220E0F0E75B',
 'date': datetime.datetime(2018, 4, 8, 9, 50, 29, 27000),
 'footprint': 'POLYGON((17.718815262801492 48.720927698149374,19.208284435787206 48.676094448248996,19.1282606560594 47.69085106573984,17.667009418148023 47.73416799677996,17.718815262801492 48.720927698149374))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('2e938509-84f3-4ae1-b2c4-0da5680c9588')/$value",
 'path': '../data/S2B_MSIL2A_20180408T095029_N0207_R079_T33UYP_20180408T115858.zip',
 'downloaded_bytes': 1203884722}

In [129]:
ll -h ../data/

total 1,5G
drwxr-xr-x 2 adrian 4,0K 15 apr 00:37 S2B_MSIL1C_20170720T090559_N0205_R050_T33PZK_20170720T092114_PROCESSED/
drwxr-xr-x 3 adrian 4,0K 11 apr 17:32 S2B_MSIL1C_20170720T090559_N0205_R050_T33PZK_20170720T092114.SAFE/
-rw-r--r-- 1 adrian 313M 11 apr 16:43 S2B_MSIL1C_20170720T090559_N0205_R050_T33PZK_20170720T092114.zip
drwxr-xr-x 2 adrian 4,0K 15 apr 13:47 S2B_MSIL2A_20180408T095029_N0207_R079_T33UYP_20180408T115858_PROCESSED/
drwxr-xr-x 3 adrian 4,0K 15 apr 00:18 S2B_MSIL2A_20180408T095029_N0207_R079_T33UYP_20180408T115858.SAFE/
-rw-r--r-- 1 adrian 1,2G 15 apr 00:14 S2B_MSIL2A_20180408T095029_N0207_R079_T33UYP_20180408T115858.zip
